In [1]:
import sys

app = "/app"
if app not in sys.path:
    sys.path.append(app)
sys.path

['/app/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/opt/venv/lib/python3.10/site-packages',
 '/app']

In [2]:
!pwd

/app/notebooks


In [3]:
!ls /data/emboj_abstracts/train

examples.txt


In [4]:
from datasets import load_dataset

dataset = load_dataset(
    "text",
    data_dir="/data/emboj_abstracts/",
    data_files={'train': 'train/examples.txt', 'test': 'test/examples.txt'},
)
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1802
    })
})

In [5]:
MODEL_NAME = "roberta-base"

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"tokenizer vocab size: {tokenizer.vocab_size}")
print(f"max length: {tokenizer.max_model_input_sizes[MODEL_NAME]}")

tokenizer vocab size: 50265
max length: 512


In [7]:
# tokenize the dataset
def tokenization(examples):
    return tokenizer(
    examples["text"],
        max_length=64, #tokenizer.max_model_input_sizes[MODEL_NAME],
        truncation=True,
        return_special_tokens_mask=True,
    )
tokenized = dataset.map(tokenization, batched=True)
tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 1802
    })
})

In [8]:
tokenized = tokenized.remove_columns(["text"])
tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 12881
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 1802
    })
})

In [9]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [17]:
from transformers import (
    RobertaForMaskedLM,
    AutoConfig
)
# from src.models.modeling_dendroberta import RobertaForMaskedLM

In [18]:
config = AutoConfig.from_pretrained(MODEL_NAME)
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [19]:
config.num_hidden_layers=2 # 8
# config.intermediate_size=128
config.max_position_embeddings=66

In [20]:
model = RobertaForMaskedLM(config=config)
model_size = sum(t.numel() for t in model.parameters())
print(f"Model size: {model_size/1000**2:.1f}M parameters")

Model size: 53.5M parameters


In [21]:
from transformers import Trainer, TrainingArguments
from datetime import datetime

args = TrainingArguments(
    output_dir="/models",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=400,
    prediction_loss_only=True,
    logging_steps=1000,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=False,
    logging_dir = f"/runs/lm-dendroberta-{datetime.now().isoformat().replace(':','-')}"
)

In [22]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized["test"],
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [23]:
trainer.train()

Step,Training Loss,Validation Loss
400,No log,6.971689
800,No log,6.644624
1200,7.210500,6.563231
1600,7.210500,6.425615
2000,6.457000,6.334223


TrainOutput(global_step=2010, training_loss=6.830868488520532, metrics={'train_runtime': 570.5206, 'train_samples_per_second': 112.888, 'train_steps_per_second': 3.523, 'total_flos': 365869983942144.0, 'train_loss': 6.830868488520532, 'epoch': 4.99})

In [24]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [38]:
# launch tensorboard
%tensorboard --logdir /runs

Reusing TensorBoard on port 6007 (pid 1453), started 0:14:50 ago. (Use '!kill 1453' to kill it.)